<a href="https://colab.research.google.com/github/snow-ham1949/ML/blob/main/ML2022Spring_HW1_3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 1: COVID-19 Cases Prediction (Regression)**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2022-spring@googlegroups.com

I use 5 models to do ensemble learning.
Each with different parameters and methods.
1. first network + L2 regularlization + lr = 5e-5 + weight_decay=1e-5 + seed=459
2. second network + lr=0.003 + seed=459
3. first network + L2 regularlization + lr=0.003 + seed=459
4. first network + L2 regularlization + lr=0.003 + weight_decay=1e-8
5. second network + L2 regularlization + lr = 5e-5 + weight_decay=1e-8 + n_epoch=5000 + batch_size=54
6. second network + lr = 0.003 + n_epoch=5000 + batch_size=54

# Download data
If the Google Drive links below do not work, you can download data from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw1/data), and upload data manually to the workspace.

In [2]:
!gdown --id '1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS' --output covid.train.csv
!gdown --id '1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg' --output covid.test.csv

Downloading...
From: https://drive.google.com/uc?id=1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS
To: /Users/snowham/Desktop/NTU/110-2/ML/covid.train.csv
100%|██████████████████████████████████████| 2.49M/2.49M [00:01<00:00, 1.39MB/s]
Downloading...
From: https://drive.google.com/uc?id=1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg
To: /Users/snowham/Desktop/NTU/110-2/ML/covid.test.csv
100%|████████████████████████████████████████| 993k/993k [00:00<00:00, 1.05MB/s]


# Import packages

In [197]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# # For plotting learning curve
# from torch.utils.tensorboard import SummaryWriter

# # Sklearn
# from sklearn.feature_selection import f_regression
# from sklearn.feature_selection import SelectKBest

model_num = 0

In [198]:
with open('./covid.train.csv', 'r') as fp:
    data = list(csv.reader(fp))
    data = np.array(data[1:])[:, 1:].astype(float) # cleaify data

# data = preprocessing.normalize(data)

In [199]:
cor_vec   = np.corrcoef(data, rowvar=False)[:, -1]
high_cols = np.where(abs(cor_vec) > 0.8)
high_cols

(array([ 37,  38,  39,  40,  52,  53,  54,  55,  56,  68,  69,  70,  71,
         72,  84,  85,  86,  87,  88, 100, 101, 102, 103, 104, 116]),)

# Some Utility Functions

You do not need to modify this part.

In [200]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Dataset

In [201]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)


  # Neural Network Model
Try out different model architectures by modifying the class below.

In [202]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        # self.layers = nn.Sequential(
        #     nn.Linear(input_dim, 128),
        #     nn.ReLU(),
        #     nn.Linear(128, 64),
        #     nn.ReLU(),
        #     nn.Linear(64, 32),
        #     nn.ReLU(),
        #     nn.Linear(32, 8),
        #     nn.ReLU(),
        #     nn.Linear(8, 4),
        #     nn.ReLU(),
        #     nn.Linear(4, 1)
        # )
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )
        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x
    
    def cal_loss(self, pred, target):
      # Use L2 regularization
      # seed = 12
      # np.random.seed(seed)
      # delta=np.random.normal(loc=0, scale=0.000001)
      # l2_lambda = 0.0001 + delta
      # l2_reg = 0

      # for param in model.parameters():
      #   l2_reg += torch.sum(param.pow(2))

      # loss = self.criterion(pred, target) + l2_lambda * l2_reg

      # return loss
    #   l1_lambda = 1e-5 + delta
    #   l1_reg = torch.tensor(0.).to(device)
    #   for param in model.parameters():
    #         l1_reg += torch.sum(torch.abs(param))
    #   loss = self.criterion(pred, target) + l1_lambda * l1_reg

    #   return loss
      return self.criterion(pred, target)

# Feature Selection
Choose features you deem useful by modifying the function below.

In [203]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [37,  38,  39,  40,  52,  53,  54,  55,  56,  68,  69,  70,  71,
        72,  84,  85,  86,  87,  88, 100, 101, 102, 103, 104]
        # TODO: Select suitable feature columns.
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# Training Loop

In [204]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    # optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9) 
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    # optimizer = torch.optim.LBFGS(model.parameters(), lr=config['learning_rate'], max_iter=config['max_iter'])

    # writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x) 
            mse_loss = model.cal_loss(pred, y);
            mse_loss.backward();            
            # loss = criterion(pred, y)
            # loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(mse_loss.detach().cpu().item())
            
            # # Display current epoch number and loss on tqdm progress bar.
            # train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            # train_pbar.set_postfix({'mse_loss': mse_loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        # writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        # val_loss = 0
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                # loss = criterion(pred, y)
                val_mse_loss = model.cal_loss(pred, y)
            loss_record.append(val_mse_loss.detach().cpu().item())

            # loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(valid_loader.dataset)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        # writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [205]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 459,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features. False: use my own features
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.            
    'batch_size': 54, 
    'learning_rate': 0.003,              
    'early_stop': 400,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model5.ckpt',  # Your model will be saved here.
    'weight_decay': 1e-8,
    'max_iter': 40
}

# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [206]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days) 
# test_data size: 1078 x 117 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid.train.csv').values, pd.read_csv('./covid.test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (2160, 118) 
valid_data size: (539, 118) 
test_data size: (1078, 117)
number of features: 24


# Start training!

In [207]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

100%|██████████| 40/40 [00:00<00:00, 2916.31it/s]


Epoch [1/5000]: Train loss: 26.8228, Valid loss: 0.1952
Saving model with loss 0.195...


100%|██████████| 40/40 [00:00<00:00, 2938.17it/s]


Epoch [2/5000]: Train loss: 8.5858, Valid loss: 0.1177
Saving model with loss 0.118...


100%|██████████| 40/40 [00:00<00:00, 2930.62it/s]


...

Epoch [1543/5000]: Train loss: 1.0827, Valid loss: 0.0210


100%|██████████| 40/40 [00:00<00:00, 3000.70it/s]


Epoch [1544/5000]: Train loss: 1.0738, Valid loss: 0.0212


100%|██████████| 40/40 [00:00<00:00, 3010.23it/s]

Epoch [1545/5000]: Train loss: 1.0645, Valid loss: 0.0210

Model is not improving, so we halt the training session.


# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [208]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred5.csv')         

100%|██████████| 20/20 [00:00<00:00, 8048.94it/s]


In [209]:
def read_prediction(name : str):
    name += '.csv'
    data = pd.read_csv(name, header=None).iloc[1:, 1:]
    data = np.array(data, dtype=np.float64).squeeze()
    return data

In [211]:
result = []
ensemble_num = 6

for i in range(0, ensemble_num):
    name = 'pred' + str(int(i))
    result.append(list(read_prediction(name)))


result = np.array(result)

res = result[0]
for i in range(1, ensemble_num):
    res += result[i]

res /= (ensemble_num)
res = list(res)

with open('result.csv', 'w') as fp:
    fp.write('id,tested_positive\n')
    for i, p in enumerate(res):
        fp.write('{},{}\n'.format(i, p))


# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)